In [1]:
"""
Simple Python script to render Apple Health data from Auto Exports
"""
from operator import truediv
import pandas as pd
import numpy as np
from datetime import datetime
import boto3
import os
import yaml
import flatdict

from ics import Calendar, Event

In [2]:

# %%
def ts_to_dt(ts):
    return datetime.fromtimestamp(ts)

def process_health_data(file):
    """
    Create [date, source] columns from files read in.
    :param file: as exported by Auto Health Export / Autosleep
    """
    df = pd.read_csv(file, sep = ',')
    if len(df.columns) > 1:
        print(f'Processing: {file.name}')
        df['creation_date'] = ts_to_dt(file.stat().st_atime)
        df['filename'] = file.name

        return df

def read_raw_files(str_path):
    """
    Read all files in a directory and return a dataframe.
    :param str_path: directory path as type string
    """
    df_health = pd.DataFrame()
    df_sleep = pd.DataFrame()
    # valid_files = ['HealthAutoExport', 'AutoSleep']
    print('Reading files..')
    file_list = os.scandir(str_path)
    csv_files = [f for f in file_list if f.name.endswith('.csv')]

    for i in csv_files:
        df_tmp = process_health_data(i)
        if i.name.startswith('HealthAutoExport'):
            df_health = pd.concat([df_health, df_tmp])
        elif i.name.startswith('AutoSleep'):
            df_sleep = pd.concat([df_sleep, df_tmp])

    return df_health, df_sleep


# %% [markdown]
# ### Transformations

# %% [markdown]
# Functions to cleanse the data
# - Rename columns
# - Dedupe values
# - Cleanse trim all values to closest integer except for sleep and weight
# - Create the following columns
#   - `Calories`


# %%
def update_columns(df, col_map):
    """
    Rename columns for easier reference
    Styling follows lowercase and no units with spaces being replaced by _
    """

    df.rename(columns=col_map, inplace=True)

    # fill in values
    df = df.replace(r'^\s+$', np.nan, regex=True)

    # convert column types
    df['date'] = pd.to_datetime(df['date']).dt.date

    # force apply float64 type for weight
    df['weight'] = df['weight'].astype(float)

    # Update column types
    df['calories'] = df['carbs'] * 4 + df['fat'] * 9 + df['protein'] * 4
    df['sleep_eff'] = df['sleep_asleep'] / df['sleep_in_bed'] * 100
    df['sleep_eff'] = df['sleep_eff'].fillna(0)
    df['sleep_eff'] = df['sleep_eff'].astype('int64')

    # Create boolean for beating threshold
    df['exercise'] = [1 if x > 30 else 0 for x in df['exercise_mins'].fillna(0)]
    df['mindful'] = [1 if x > 5 else 0 for x in df['mindful_mins'].fillna(0)]

    return df

def round_df(df):
    """
    Round all numerical columns to closest integer except for one d.p. cols
    Replaces all NaN with null
    """
    one_dp_cols = ['sleep_asleep', 'sleep_in_bed', 'weight']
    for i in df.columns:
        if df[i].dtypes == 'float64':
            if i in one_dp_cols:
                df[i] = df[i].round(1)
            else:
                df[i] = np.floor(pd.to_numeric(df[i], errors= 'coerce')).astype('Int64')

    return df

def dedup_df(df):
    """
    Remove duplicates ordering by 'date' and 'creation_date' and then keep only the latest
    """
    df_sort = df.sort_values(['date', 'creation_date'], ascending= True)
    df_dedup = df_sort.drop_duplicates(subset = 'date', keep = 'last')

    return df_dedup

# %%
def create_description_cols(df, is_autosleep=False):
    """
    Create description columns for the generating events
    Converts events into boolean
    """
    print("Creating description columns for calendar events")
    # cleansing Autosleep data
    if is_autosleep:
        print("Updating sleep statistics")
        df['description_sleep'] =  df.agg(lambda x: f"{x['deep']} / {int(x['efficiency'])}%]\r\n(🌒 {x['bedtime']} /🌞 {x['waketime']})", axis=1)

        df['sleep'] = df.agg(lambda x: f"{x['asleep']}", axis = 1)

        return df
    # cleansing Apple Health Data
    else:
        for i in df.columns:
            if df[i].dtypes == 'float64':
                df[i] = df[i].apply(lambda x: f"{x:,.1f}")
            elif df[i].dtypes in ('int64', 'Int64'):
                df[i] = df[i].map('{:,.0f}'.format)

        print("Creating description columns")

        df['description_food'] = [f"({a}C/{b}P/{c}F)" for a,b,c in zip(df['carbs'], df['protein'], df['fat'])]

        df['food'] = [f"{a} calories {b}" for a,b in zip(df['calories'], df['description_food']) ]
        df['activity'] = [f"{a} steps" for a in df['steps']]
        df['sleep'] = [f"{a} h ({b} % eff.)" for a,b in zip(df['sleep_asleep'], df['sleep_eff'])]

        # Cleanse data
        df['sleep'] = df['sleep'].replace('nan h (0% eff.)', 'No sleep data.')

        return df

def convert_autosleep_time(time, is_24h=False):
    """
    Converts time from a string; stripping the date and adding the AM / PM / hours and minutes
    """
    time_dt = time.split(" ")[-1][:5]

    if is_24h:
        time_dt = datetime.strptime(time_dt, "%H:%M")
        time_dt = time_dt.strftime("%-I:%M %p")
    else:
        hours = int(time_dt.split(":")[0])
        min = int(time_dt.split(":")[1])
        time_dt = f"{hours} h {min} m"

    return time_dt

def etl_autosleep_data(df):
    """
    Cleans autosleep data into correct formatting
    """
    #  Clean up the time columns with either 12 h format (AM / PM) or with hours and minutes
    time_dict = {
        '24h': ['bedtime', 'waketime'],
        'hrs': ['asleep', 'deep']
    }
    for time_type, time_cols in time_dict.items():
        is_24h = 0
        for time_col in time_cols:
            if time_type == '24h': is_24h = 1
            df[time_col] = df[time_col].apply(lambda x: convert_autosleep_time(x, is_24h))

    # Collect the date
    df['date'] = df['ISO8601'].apply(lambda x: datetime.strptime(x.split("T")[0], '%Y-%m-%d').date())

    df = create_description_cols(df, is_autosleep=True)

    # Remove duplicates
    df = dedup_df(df)

    return df


def get_config(config_file):
    """
    Generate configs are read from config.yml
    If no values defined, return as current working directory
    """
    config = yaml.load(open(config_file, "r"),  Loader=yaml.FullLoader)
    config = flatdict.FlatDict(config, delimiter = '.')
    for k, v in config.items():
        if not k.startswith('type'):
            if v == "": config[k] = os.getcwd()

    return config


In [3]:
config = get_config('config.yml')
input_path = config.get('input.raw_path')
output_path = config.get('output.raw_path')
output_cal = config.get('output.calendar_path')
output_file_name = config.get('output.file_name')
region = config.get('type.region')
col_map = config.get('col_map')

In [7]:
df, df_sleep = read_raw_files(input_path)

Reading files..
Processing: HealthAutoExport-2022-06-23-2022-06-29 Data.csv
Processing: HealthAutoExport-2022-07-06-2022-07-06 Data.csv
Processing: HealthAutoExport-2022-07-07-2022-07-07 Data.csv
Processing: AutoSleep-20220601-to-20220630.csv
Processing: HealthAutoExport-2022-07-04-2022-07-10 Data.csv
Processing: HealthAutoExport-2022-07-05-2022-07-11 Data.csv
Processing: HealthAutoExport-2022-06-28-2022-06-28 Data.csv
Processing: HealthAutoExport-2022-07-01-2022-07-01 Data.csv
Processing: HealthAutoExport-2022-06-01-2022-06-27 Data.csv
Processing: HealthAutoExport-2022-07-09-2022-07-15 Data.csv
Processing: HealthAutoExport-2022-07-07-2022-07-13 Data.csv
Processing: HealthAutoExport-2022-07-04-2022-07-04 Data.csv
Processing: HealthAutoExport-2022-06-30-2022-06-30 Data.csv
Processing: AutoSleep-20220601-to-20220710.csv
Processing: AutoSleep-20220701-to-20220716.csv
Processing: HealthAutoExport-2022-07-09-2022-07-09 Data.csv
Processing: HealthAutoExport-2022-07-03-2022-07-09 Data.csv
Pro

In [9]:
df = update_columns(df, col_map)

In [10]:
df

,date,carbs,protein,sleep_asleep,sleep_in_bed,steps,fat,weight,creation_date,filename,exercise_mins,mindful_mins,calories,sleep_eff,exercise,mindful
0,2022-06-23,267.93,198.35,9.266667,9.300000,13519,60.68,74.7,2022-07-16 21:28:59.486837,HealthAutoExport-2022-06-23-2022-06-29 Data.csv,NaN,NaN,2411.24,99,0,0
1,2022-06-24,266.80,166.45,9.100000,9.100000,13092,69.44,NaN,2022-07-16 21:28:59.486837,HealthAutoExport-2022-06-23-2022-06-29 Data.csv,NaN,NaN,2357.96,100,0,0
2,2022-06-25,327.51,163.98,7.466667,8.745598,11068,76.25,NaN,2022-07-16 21:28:59.486837,HealthAutoExport-2022-06-23-2022-06-29 Data.csv,NaN,NaN,2652.21,85,0,0
3,2022-06-26,242.65,249.75,7.533333,8.033333,13934,93.84,74.5,2022-07-16 21:28:59.486837,HealthAutoExport-2022-06-23-2022-06-29 Data.csv,NaN,NaN,2814.16,93,0,0
4,2022-06-27,341.42,179.76,8.300000,9.900481,11200,54.18,NaN,2022-07-16 21:28:59.486837,HealthAutoExport-2022-06-23-2022-06-29 Data.csv,NaN,NaN,2572.34,83,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2022-07-08,289.23,176.42,7.883333,7.913608,12787,81.35,74.6,2022-07-16 21:29:00.097624,HealthAutoExport-2022-07-03-2022-07-09 Data.csv,NaN,NaN,2594.75,99,0,0
6,2022-07-09,292.89,169.87,NaN,NaN,20532,81.78,74.0,2022-07-16 21:29:00.097624,HealthAutoExport-2022-07-03-2022-07-09 Data.csv,NaN,NaN,2587.06,0,0,0
0,2022-06-26,242.65,249.75,7.533333,8.033333,13934,93.84,74.5,2022-07-16 21:28:59.598604,HealthAutoExport-2022-06-26-2022-06-26 Data.csv,NaN,NaN,2814.16,93,0,0
0,2022-06-27,341.42,179.76,8.300000,9.900481,11200,54.18,NaN,2022-07-16 21:28:59.708918,HealthAutoExport-2022-06-27-2022-06-27 Data.csv,NaN,NaN,2572.34,83,0,0


In [11]:
list_events = ['mindful', 'exercise', 'activity']

In [12]:
df, df_sleep = read_raw_files(input_path)
df = update_columns(df, col_map)
df = round_df(df)
df = dedup_df(df)
df = create_description_cols(df)
df = df.reset_index(drop=True)

Reading files..
Processing: HealthAutoExport-2022-06-23-2022-06-29 Data.csv
Processing: HealthAutoExport-2022-07-06-2022-07-06 Data.csv
Processing: HealthAutoExport-2022-07-07-2022-07-07 Data.csv
Processing: AutoSleep-20220601-to-20220630.csv
Processing: HealthAutoExport-2022-07-04-2022-07-10 Data.csv
Processing: HealthAutoExport-2022-07-05-2022-07-11 Data.csv
Processing: HealthAutoExport-2022-06-28-2022-06-28 Data.csv
Processing: HealthAutoExport-2022-07-01-2022-07-01 Data.csv
Processing: HealthAutoExport-2022-06-01-2022-06-27 Data.csv
Processing: HealthAutoExport-2022-07-09-2022-07-15 Data.csv
Processing: HealthAutoExport-2022-07-07-2022-07-13 Data.csv
Processing: HealthAutoExport-2022-07-04-2022-07-04 Data.csv
Processing: HealthAutoExport-2022-06-30-2022-06-30 Data.csv
Processing: AutoSleep-20220601-to-20220710.csv
Processing: AutoSleep-20220701-to-20220716.csv
Processing: HealthAutoExport-2022-07-09-2022-07-09 Data.csv
Processing: HealthAutoExport-2022-07-03-2022-07-09 Data.csv
Pro

In [13]:
df

,date,carbs,protein,sleep_asleep,sleep_in_bed,steps,fat,weight,creation_date,filename,exercise_mins,mindful_mins,calories,sleep_eff,exercise,mindful,description_food,food,activity,sleep
0,2022-06-01,279,104,9.0,9.3,"10,957",61,nan,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,085",96,0,0,(279C/104P/61F),"2,085 calories (279C/104P/61F)","10,957 steps",9.0 h (96 % eff.)
1,2022-06-02,308,146,nan,nan,"10,639",53,72.4,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,300",0,0,0,(308C/146P/53F),"2,300 calories (308C/146P/53F)","10,639 steps",nan h (0 % eff.)
2,2022-06-03,282,128,7.6,7.6,"13,124",52,nan,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,119",100,0,0,(282C/128P/52F),"2,119 calories (282C/128P/52F)","13,124 steps",7.6 h (100 % eff.)
3,2022-06-04,289,182,8.7,8.7,"9,345",86,nan,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,671",100,0,0,(289C/182P/86F),"2,671 calories (289C/182P/86F)","9,345 steps",8.7 h (100 % eff.)
4,2022-06-05,260,148,nan,nan,"8,875",78,nan,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,341",0,0,0,(260C/148P/78F),"2,341 calories (260C/148P/78F)","8,875 steps",nan h (0 % eff.)
5,2022-06-06,288,185,9.6,10.1,"11,716",64,nan,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,474",94,0,0,(288C/185P/64F),"2,474 calories (288C/185P/64F)","11,716 steps",9.6 h (94 % eff.)
6,2022-06-07,249,162,nan,nan,"14,449",77,nan,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,346",0,0,0,(249C/162P/77F),"2,346 calories (249C/162P/77F)","14,449 steps",nan h (0 % eff.)
7,2022-06-08,284,147,8.2,9.2,"14,780",68,nan,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,347",88,0,0,(284C/147P/68F),"2,347 calories (284C/147P/68F)","14,780 steps",8.2 h (88 % eff.)
8,2022-06-09,326,148,6.3,7.0,"13,597",45,nan,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,311",90,0,0,(326C/148P/45F),"2,311 calories (326C/148P/45F)","13,597 steps",6.3 h (90 % eff.)
9,2022-06-10,305,154,8.2,9.4,"8,614",71,nan,2022-07-16 21:28:59.362675,HealthAutoExport-2022-06-01-2022-06-27 Data.csv,<NA>,<NA>,"2,482",86,0,0,(305C/154P/71F),"2,482 calories (305C/154P/71F)","8,614 steps",8.2 h (86 % eff.)


In [14]:
df_events = df[['date', 'food', 'activity', 'sleep', 'exercise', 'mindful']].melt(
        id_vars = ['date'],
        value_vars = ['food', 'activity', 'sleep', 'exercise', 'mindful'],
        var_name = 'event_type',
        value_name = 'event_name'
    )

In [15]:
df_events.query('event_type in @list_events')

,date,event_type,event_name
45,2022-06-01,activity,"10,957 steps"
46,2022-06-02,activity,"10,639 steps"
47,2022-06-03,activity,"13,124 steps"
48,2022-06-04,activity,"9,345 steps"
49,2022-06-05,activity,"8,875 steps"
...,...,...,...
220,2022-07-11,mindful,1
221,2022-07-12,mindful,0
222,2022-07-13,mindful,1
223,2022-07-14,mindful,0
